In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import svm
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

##init X train         
X = pd.read_csv("train_features.csv")

##init y train
y = pd.read_csv("train_labels.csv")

##init X train
X_test = pd.read_csv("test_features.csv")

##rearange 12 samples to one, because it's a time series
def widen(X, time_series_amount):
    return pd.DataFrame(data=X.values.reshape(int(X.shape[0] / time_series_amount), X.shape[1] * time_series_amount))

def fill_time_series(X, time_series_amount=12, median=None):
    
    if median is None: 
        median = X.median()   
    
    X_filled = pd.DataFrame(columns=X.columns)
    for i in range(0, X.shape[0], time_series_amount):
         series = X.iloc[i:i + time_series_amount,:].mean()
         X_filled_12 = X.iloc[i:i + time_series_amount,:].fillna(series)
         X_filled = X_filled.append(X_filled_12)
        
    return X_filled.fillna(median), median

def drop_features(X, featureNames):
    return X.drop(featureNames, axis = 1)


def write_out(target_label_predictions, target_label, y_df, folder_path):
    df = pd.DataFrame(columns=y.columns.values)
    df[target_label] = target_label_predictions
    df.to_csv(folder_path + "/" + label_name + ".csv", index=False)
    

def pp_X_and_X_test(X, X_test, drop_features_list, time_series_amount=12):
    ##train set pp
    X = drop_features(X, drop_features_list)
    X, median = fill_time_series(X, time_series_amount)
    scaler = StandardScaler().fit(X)
    X = pd.DataFrame(data=scaler.transform(X), columns=X.columns)
    X = widen(X, time_series_amount)
    
    ##test set pp
    X_test = drop_features(X_test, drop_features_list)
    X_test, _ = fill_time_series(X_test, time_series_amount, median)
    X_test = pd.DataFrame(data=scaler.transform(X_test), columns=X_test.columns)
    X_test = widen(X_test, time_series_amount)
    
    return X, X_test
    
def train_GB_w_writeout_train_and_test(X, y, target_label, X_test, random_state):
    print("start calculation: " + target_label)
    gb = ensemble.GradientBoostingClassifier(random_state = random_state).fit(X, y[target_label])
    
    ##compare with train_labels & save file
    y_predict = gb.predict_proba(X)
    y_true = y[target_label]
    print(roc_auc_score(y_true, y_predict))
    write_out(y_predict, target_label, y, "predict_train_labels")
    
    ##predict and save to file
    y_test_predict = gb.predict_proba(X_test)
    write_out(y_test_predict, target_label, y, "predict_test_labels")
    print("finish calculation: " + target_label)
    
    return gb
    


In [ ]:
X_PP, X_test_PP = pp_X_and_X_test(X, X_test, ["pid", "Time","Age"])

In [6]:
X_PP = widen(X_PP, 12)

In [9]:
##14:37
##14:44

,pid,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,33.0,32.2,12.0,2.1,36.00,8.700000,24.000000,-2.000000,16.0,...,72.0,100.000000,0.3,114.0,24.60,94.0,0.8,0.15,142.0,7.33
1,1,33.0,32.2,12.0,2.1,36.00,8.566667,25.333333,-2.000000,16.0,...,72.0,100.000000,0.3,112.0,23.20,99.0,0.8,0.15,125.0,7.33
2,1,33.0,32.2,12.0,2.1,36.00,8.566667,25.333333,0.000000,18.0,...,72.0,100.000000,0.3,112.0,23.20,92.0,0.8,0.15,110.0,7.37
3,1,33.0,32.2,12.0,2.1,37.00,8.566667,25.333333,0.000000,18.0,...,72.0,100.000000,0.3,112.0,23.20,88.0,0.8,0.15,104.0,7.37
4,1,33.0,32.2,12.0,2.1,36.75,8.566667,25.333333,-0.666667,18.0,...,72.0,100.000000,0.3,112.0,22.40,81.0,0.8,0.15,100.0,7.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,9999,33.0,36.4,30.0,2.1,36.00,9.450000,25.000000,-1.000000,17.0,...,72.0,97.571429,0.3,107.0,29.45,80.0,0.8,0.15,110.0,7.37
227936,9999,33.0,36.4,30.0,2.1,36.00,9.450000,25.000000,-1.000000,15.0,...,72.0,97.571429,0.3,107.0,29.45,83.0,0.8,0.15,123.0,7.37
227937,9999,33.0,36.4,30.0,2.1,36.00,9.450000,25.000000,-1.000000,22.0,...,72.0,98.000000,0.3,107.0,29.45,80.0,0.8,0.15,138.0,7.37
227938,9999,33.0,36.4,30.0,2.1,36.00,10.200000,25.000000,-1.000000,22.0,...,72.0,98.000000,0.3,107.0,31.00,75.0,0.8,0.15,125.0,7.37


In [141]:
train_GB_w_writeout_train_and_test(arr1, y, X_test, 0, ["c"], 2)

,0,1,2,3
0,1.643168,0.0,0.547723,0.0
1,0.000000,0.0,0.000000,0.0
2,-0.547723,0.0,-1.643168,0.0


In [10]:
X

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,3,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,...,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,1,4,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
2,1,5,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
3,1,6,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
4,1,7,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,9999,8,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN,110.0,NaN
227936,9999,9,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,123.0,NaN
227937,9999,10,85.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,98.0,NaN,NaN,NaN,80.0,NaN,NaN,138.0,NaN
227938,9999,11,85.0,NaN,NaN,NaN,NaN,NaN,10.2,NaN,...,NaN,98.0,NaN,NaN,31.0,75.0,NaN,NaN,125.0,NaN


In [89]:
time_series_amount = 2
X = X.copy()
for i in range(0, X.shape[0], time_series_amount):
        series = X.iloc[i:i + time_series_amount,:].mean()
        X.iloc[i:i + time_series_amount,:].fillna(series)

In [96]:
series = X.iloc[0:0 + time_series_amount,:].mean()
series

a    NaN
b    2.0
c    3.0
dtype: float64

In [97]:
X.iloc[0:0 + time_series_amount,:].fillna(series)

,a,b,c
0,NaN,2.0,3.0
1,NaN,2.0,3.0


In [142]:
arr = None

In [143]:
arr

In [147]:
if arr is None: print("hello")

hello
